<a href="https://colab.research.google.com/github/hossein20s/tutorial/blob/master/Restore_LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf

print(tf.__version__)

1.13.1


In [7]:
import sys
sys.path.append('/content/gdrive/My Drive/python-lib')

from google.colab import drive
drive.mount('/content/gdrive')

persist_path = "gdrive/My Drive/models/modeling_stock_market"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from collections import Counter
import numpy as np
import utils as utl
import pandas as pd

data = pd.read_csv("https://github.com/GarrettHoffman/lstm-oreilly/raw/master/data/StockTwits_SPY_Sentiment_2017.gz",
                   encoding="utf-8",
                   compression="gzip",
                   index_col=0)

# get messages and sentiment labels
messages = data.message.values
labels = data.sentiment.values

# make everything lower case and remove punctuation
messages = np.array([utl.preprocess_ST_message(message) for message in messages])

# keeping a collection of our full vocabularly and creating a mapping of each word to a unique index

full_lexicon = " ".join(messages).split()
vocab_to_int, int_to_vocab = utl.create_lookup_tables(full_lexicon)
messages_lens = Counter([len(x) for x in messages])
messages, labels = utl.drop_empty_messages(messages, labels)

messages = utl.encode_ST_messages(messages, vocab_to_int)
labels = utl.encode_ST_labels(labels)
messages = utl.zero_pad_messages(messages, seq_len=244)

train_x, val_x, test_x, train_y, val_y, test_y = utl.train_val_test_split(messages, labels, split_frac=0.80)

In [9]:
batch_size = 256
vocab_size = len(vocab_to_int) + 1 #add one for padding
embed_size = 300
lstm_sizes = [128, 64]
learning_rate = 0.1


def test_network(model_dir, batch_size, test_x, test_y):
    
    inputs_, labels_, keep_prob_ = model_inputs()
    embed = build_embedding_layer(inputs_, vocab_size, embed_size)
    initial_state, lstm_outputs, lstm_cell, final_state = build_lstm_layers(lstm_sizes, embed, keep_prob_, batch_size)
    predictions, loss, optimizer = build_cost_fn_and_opt(lstm_outputs, labels_, learning_rate)
    accuracy = build_accuracy(predictions, labels_)
    
    saver = tf.train.Saver()
    
    test_acc = []
    with tf.Session() as sess:
        saver = tf.train.import_meta_graph(model_dir + '/sentiment.ckpt.meta')
        saver.restore(sess,tf.train.latest_checkpoint(model_dir))
        test_state = sess.run(lstm_cell.zero_state(batch_size, tf.float32))
        for ii, (x, y) in enumerate(utl.get_batches(test_x, test_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob_: 1,
                    initial_state: test_state}
            batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
            test_acc.append(batch_acc)
        print("Test Accuracy: {:.3f}".format(np.mean(test_acc)))
        


#from google.colab import files
#files.download( modelSavingPath ) 

with tf.Graph().as_default():
    test_network(persist_path, batch_size, test_x, test_y)
   


INFO:tensorflow:Restoring parameters from gdrive/My Drive/models/modeling_stock_market/sentiment.ckpt
Test Accuracy: 0.614
